Predict script for Merge model, predict the D and R matrice, and visualize resule.

In [1]:
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from modules.merge_modules import MergeModel
import torch.backends.cudnn as cudnn
import torch
import json
from dataset.dataset import ImageDataset
from PIL import Image
import cv2
import numpy as np

In [4]:
# init the Merge model
if torch.cuda.is_available():
    print('Init model with CUDA')
    net = MergeModel(3).cuda()
    cudnn.benchmark = True
    cudnn.deterministic = True
    net = torch.nn.DataParallel(net).cuda()
else:
    print('Init model with CPU')
    net = MergeModel(3)
    net = torch.nn.DataParallel(net)

Init model with CPU


In [5]:
# load saved checkpoint 
PATH = '/Users/longhoang/Developer/table-reg/pretrain-models/split-merge/merge_cp.pth'
if torch.cuda.is_available():
    net.load_state_dict(torch.load(PATH))
else:
    net.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))

In [6]:
# change the model to eval mode
net.eval()

DataParallel(
  (module): MergeModel(
    (sfcn): SharedFCN(
      (conv): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 18, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
          (1): ReLU(inplace=True)
        )
        (1): Sequential(
          (0): Conv2d(18, 18, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
          (1): ReLU(inplace=True)
        )
        (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
        (3): Sequential(
          (0): Conv2d(18, 18, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
          (1): ReLU(inplace=True)
        )
        (4): Sequential(
          (0): Conv2d(18, 18, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
          (1): ReLU(inplace=True)
        )
        (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      )
    )
    (rpn1): ProjectionNet(
      (conv_branch1): Sequent

In [7]:
# init dataset
DATA_PATH = '/Users/longhoang/Developer/table-reg/data/scitsr-split-train/val/img'
LABEL_PATH = '/Users/longhoang/Developer/table-reg/data/scitsr-split-train/val/label/merge-label.json'

with open(LABEL_PATH, 'r') as f:
    labels = json.load(f)

dataset = ImageDataset(DATA_PATH, labels, 8, scale=1,mode='merge')

len(dataset), len(labels)

(1971, 1971)

In [8]:
index = 0
img, label, arc = dataset[index]
index += 1

In [9]:
len(arc[0]), len(arc[1]), img.shape

(237, 424, torch.Size([3, 237, 424]))

In [10]:
h_line = arc[0]
h_line

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [11]:
# predict 
input_img = img.unsqueeze(0)
arc_c = [[torch.Tensor([y]) for y in x] for x in arc]
pred = net(input_img,arc_c)
u,d,l,r = pred # up, down, left, right
# calculate D and R matrice, 
D = 0.5 * u[:, :-1, :] * d[:, 1:, :] + 0.25 * (u[:, :-1, :] + d[:, 1:, :])
R = 0.5 * r[:, :, :-1] * l[:, :, 1:] + 0.25 * (r[:, :, :-1] + l[:, :, 1:])
D = D[0].detach().cpu().numpy()
R = R[0].detach().cpu().numpy()
D[D>0.5] = 1
D[D<=0.5] = 0
R[R>0.5] = 1
R[R<=0.5] = 0

rows, columns = arc
h,w = img[2].shape
rows = [round(h*x) for x in rows]
columns = [round(w*x) for x in columns]
rows = [0] + rows + [h]
columns = [0] + columns +[w]

# draw lines on the original image
draw_img = img[2].numpy()*255.
draw_img = cv2.cvtColor(draw_img, cv2.COLOR_GRAY2RGB)
for i in range(R.shape[0]):
    for j in range(R.shape[1]):
        if R[i,j] == 0:
            pts1 = (columns[j+1],rows[i])
            pts2 = (columns[j+1],rows[i+1])
            draw_img = cv2.line(draw_img, pts1,pts2,(255.,0,0),2)
for i in range(D.shape[0]):
    for j in range(D.shape[1]):
        if D[i,j] == 0:
            pts1 = (columns[j],rows[i+1])
            pts2 = (columns[j+1],rows[i+1])
            draw_img = cv2.line(draw_img, pts1,pts2,(255.,0,0),2)

RuntimeError: split_with_sizes expects split_sizes have only non-negative entries, but got split_sizes=[59, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, -59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# visualize original image
Image.fromarray(img[2].numpy()*255.).convert('L')

In [ ]:
# visualize merged image
Image.fromarray(np.array(draw_img,dtype=np.uint8))